In [62]:
import os
import pandas as pd
import numpy as np
from skimage.io import imsave
from skimage.io import imread as imread_sk
from skimage.measure import regionprops_table
from tifffile import imread
import napari
import matplotlib.pyplot as plt
from skimage.segmentation import expand_labels

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [63]:
df_path = r'D:\data_analysis\Betsy_Suraj\230112_Tcell_analysis\230117_exp_summary.pkl'

dna_mask_corr_dir = r'D:\data_analysis\Betsy_Suraj\230112_Tcell_analysis\segmentation_corr_DNA_4'
cd44_mask_corr_dir = r'D:\data_analysis\Betsy_Suraj\230112_Tcell_analysis\segmentation_corr_CD44'

results_dir = r'D:\data_analysis\Betsy_Suraj\230112_Tcell_analysis\results'

In [64]:
properties = ['label','centroid','bbox','area','solidity','eccentricity','image','mean_intensity']

In [65]:
try:
    os.mkdir(results_dir)
except:
    print('Results directory exists.')

Results directory exists.


In [66]:
df = pd.read_pickle(df_path)
df

,file_name,exp_dir,im_path,slide,well,exp,marker,dye,im_round,density,donor,treatment
0,T3737_9_2_1.0.4_R000_Cy3_CycB1-AF555_FINAL_AFR...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_C...,T3737_9_2,R000,T3737_9_2_R000,CycB1,Cy3,1,150,D1,0_plated
1,T3737_9_2_1.0.4_R000_Cy5_CD8a-AF647_FINAL_AFR_...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_C...,T3737_9_2,R000,T3737_9_2_R000,CD8a,Cy5,1,150,D1,0_plated
2,T3737_9_2_1.0.4_R000_Cy7_CD44-AF750_FINAL_AFR_...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_C...,T3737_9_2,R000,T3737_9_2_R000,CD44,Cy7,1,150,D1,0_plated
3,T3737_9_2_1.0.4_R000_DAPI__FINAL_F.tif,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_D...,T3737_9_2,R000,T3737_9_2_R000,DNA_01,DAPI,1,150,D1,0_plated
4,T3737_9_2_1.0.4_R000_FITC_PCNA-AF488_FINAL_AFR...,D:\data\T3737_9_2_Final,D:\data\T3737_9_2_Final\T3737_9_2_1.0.4_R000_F...,T3737_9_2,R000,T3737_9_2_R000,PCNA,FITC,1,150,D1,0_plated
...,...,...,...,...,...,...,...,...,...,...,...,...
315,T3937_39_4.0.4_R007_Cy3_p21-AF555_FINAL_AFR_F.tif,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_Cy3...,T3937_39,R007,T3937_39_R007,p21,Cy3,4,200,D2,24_stim
316,T3937_39_4.0.4_R007_Cy5_FoxP3-AF647_FINAL_AFR_...,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_Cy5...,T3937_39,R007,T3937_39_R007,FoxP3,Cy5,4,200,D2,24_stim
317,T3937_39_4.0.4_R007_Cy7_cycB2-AF750_FINAL_AFR_...,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_Cy7...,T3937_39,R007,T3937_39_R007,cycB2,Cy7,4,200,D2,24_stim
318,T3937_39_4.0.4_R007_DAPI__FINAL_F.tif,D:\data\T3937_39_Final,D:\data\T3937_39_Final\T3937_39_4.0.4_R007_DAP...,T3937_39,R007,T3937_39_R007,DNA_04,DAPI,4,200,D2,24_stim


In [67]:
# list all experiments
exp_list = list(set(df.exp))
exp_list.sort()

# list all signals
marker_list = list(set(df.marker))
marker_list.sort()

# create a dictionary to rename columns
int_list = [f'mean_intensity-{x}' for x in range(20)]
dict = {int_list[i]: marker_list[i] for i in range(len(int_list))}

# create small df with exp info
df_slide = df.drop(['file_name','im_path','marker','dye','im_round'],axis=1)
df_slide = df_slide.drop_duplicates()

In [85]:
marker_list

['CD4',
 'CD44',
 'CD69',
 'CD8a',
 'CDC25C',
 'CDK2',
 'CycB1',
 'DNA_01',
 'DNA_02',
 'DNA_03',
 'DNA_04',
 'FoxP3',
 'PCNA',
 'cycA2',
 'cycB2',
 'p21',
 'p27',
 'pERK',
 'pRB',
 'pS6']

In [68]:
exp_list

['T3737_9_2_R000',
 'T3737_9_2_R001',
 'T3737_9_2_R002',
 'T3737_9_2_R003',
 'T3737_9_2_R004',
 'T3737_9_2_R005',
 'T3737_9_2_R006',
 'T3737_9_2_R007',
 'T3937_39_R000',
 'T3937_39_R001',
 'T3937_39_R002',
 'T3937_39_R003',
 'T3937_39_R004',
 'T3937_39_R005',
 'T3937_39_R006',
 'T3937_39_R007']

In [83]:
for exp_sel in exp_list:

    print(exp_sel)

    df_sel = df.loc[df.exp==exp_sel,:]

    # open mask images
    dna_im_name = df_sel.loc[df_sel.marker=='DNA_04','file_name'].tolist()[0]
    dna_mask_path = os.path.join(dna_mask_corr_dir,dna_im_name.replace('.tif','_maskdna4_corr.png'))

    cd44_im_name = df_sel.loc[df_sel.marker=='CD44','file_name'].tolist()[0]
    cd44_mask_path = os.path.join(cd44_mask_corr_dir,cd44_im_name.replace('.tif','_maskcd44_corr.png'))

    dna_mask_clean = imread_sk(dna_mask_path)
    cd44_mask_clean = imread_sk(cd44_mask_path)

    # open signal images
    int_im_stack = []
    for sel_marker in marker_list:

        marker_im_path = df_sel.loc[df_sel.marker==sel_marker,'im_path'].tolist()[0]

        im = imread(marker_im_path)

        int_im_stack.append(im)

    int_im_stack = np.array(int_im_stack)
    int_im_stack = np.moveaxis(int_im_stack,0,2)
    

    # calculate properties of nuc and cell
    df_dna = regionprops_table(dna_mask_clean,intensity_image=int_im_stack,properties=properties)
    df_dna = pd.DataFrame(df_dna)
    print(f'Total number of nuclear objects: {len(df_dna)}')

    df_cell = regionprops_table(cd44_mask_clean,intensity_image=int_im_stack,properties=properties)
    df_cell = pd.DataFrame(df_cell)
    print(f'Total number of cell objects: {len(df_cell)}')

    # calculate properties of cytoplasm
    inside_nuc_mask = np.zeros(dna_mask_clean.shape)
    inside_nuc_mask[dna_mask_clean == cd44_mask_clean] = dna_mask_clean[dna_mask_clean == cd44_mask_clean]
    cyto_mask = cd44_mask_clean.astype(float) - inside_nuc_mask.astype(float)
    cyto_mask[cyto_mask < 0] = 0
    cyto_mask = cyto_mask.astype('uint16')


    df_cyto = regionprops_table(cyto_mask,intensity_image=int_im_stack,properties=['label','area','mean_intensity'])
    df_cyto = pd.DataFrame(df_cyto)
    print(f'Total number of cyto objects: {len(df_cyto)}')

    # calculate properties of background
    bck_mask = expand_labels(cd44_mask_clean,distance=25)
    bck_mask_inner = expand_labels(cd44_mask_clean,distance=5)
    bck_mask = bck_mask - bck_mask_inner

    df_bck = regionprops_table(bck_mask,intensity_image=int_im_stack,properties=['label','area','mean_intensity'])
    df_bck = pd.DataFrame(df_bck)
    print(f'Total number of bck objects: {len(df_bck)}')

    # merge data frames
    df_nuc_cell = pd.merge(left=df_dna,right=df_cell,on='label',suffixes=('_nuc','_cell'))
    df_cyto_bck = pd.merge(left=df_cyto,right=df_bck,on='label',suffixes=('_cyto','_bck'))
    df_all = pd.merge(left=df_nuc_cell,right=df_cyto_bck,on='label')

    # merge with additional info about the experiment
    df_all['exp'] = exp_sel
    df_all = pd.merge(left=df_all,right=df_slide,on='exp')

    new_names = []
    for col_name in df_all.columns:

        try:
            core_name = '_'.join(col_name.split('_')[:2])
            reg_name = col_name.split('_')[-1]
            col_name = f'{dict[core_name]}_mean_{reg_name}'
        except:
            pass

        new_names.append(col_name)

    df_all.columns = new_names

    # save all df
    save_path = os.path.join(results_dir,f'{exp_sel}_all.pkl')
    df_all.to_pickle(save_path)

    # save a version without images
    df_small = df_all.drop(['image_nuc','image_cell'],axis=1)

    save_path_small = os.path.join(results_dir,f'{exp_sel}.pkl')
    df_small.to_pickle(save_path_small)

T3737_9_2_R000
Total number of nuclear objects: 9292
Total number of cell objects: 9292
Total number of cyto objects: 9279
Total number of bck objects: 9292
T3737_9_2_R001
Total number of nuclear objects: 21642
Total number of cell objects: 21642
Total number of cyto objects: 21607
Total number of bck objects: 21633
T3737_9_2_R002
Total number of nuclear objects: 15488
Total number of cell objects: 15488
Total number of cyto objects: 15454
Total number of bck objects: 15457
T3737_9_2_R003
Total number of nuclear objects: 13548
Total number of cell objects: 13548
Total number of cyto objects: 13517
Total number of bck objects: 13548
T3737_9_2_R004
Total number of nuclear objects: 9091
Total number of cell objects: 9091
Total number of cyto objects: 9084
Total number of bck objects: 9091
T3737_9_2_R005
Total number of nuclear objects: 30947
Total number of cell objects: 30947
Total number of cyto objects: 30929
Total number of bck objects: 30947
T3737_9_2_R006
Total number of nuclear obj